In [1]:
import numpy as np
import plotly.graph_objects as go

In [3]:
# Load the occupancy data from the .npy file
#occupancy_grid = np.load('./object_occupancy.npy')
occupancy_grid = np.load(r'object_occupancy.npy')
#G:\Mixed_Reality\cs\object_occupancy.npy
# Get the dimensions of the grid
grid_size = occupancy_grid.shape[0]

# Generate grid coordinates
x, y, z = np.meshgrid(np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size),
                      indexing='ij')

# Filter coordinates based on occupancy
occupied_x = x[occupancy_grid]
occupied_y = y[occupancy_grid]
occupied_z = z[occupancy_grid]
print(np.min(occupied_x), np.max(occupied_x))
print(np.min(occupied_y), np.max(occupied_y))
print(np.min(occupied_z), np.max(occupied_z))

# Create a 3D scatter plot
occupancy = go.Scatter3d(x=occupied_x, y=occupied_y, z=occupied_z, mode='markers', marker=dict(size=1))
fig = go.Figure(data=[occupancy])

# Update layout
fig.update_layout(
    title="3D Occupancy Grid",
    scene=dict(
        xaxis=dict(nticks=4, range=[-1, 1]),
        yaxis=dict(nticks=4, range=[-1, 1]),
        zaxis=dict(nticks=4, range=[-1, 1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


-0.4173228346456693 0.1811023622047243
-0.19685039370078738 0.2913385826771653
-0.6062992125984252 -0.13385826771653542


In [11]:
points1 = np.load(r'plane_samples.npy')
points2 = np.load(r'plane_samples_cs.npy')

# Plane equation coefficients
# a, b, c, d = -0.7358716726303101, -0.14783693850040436, -1, -0.013480226509273052
# -0.0742659941315651x + -0.09833580255508423y + -1z + -0.6207793951034546 = 0
# a, b, c, d = -0.0742659941315651, -0.09833580255508423, -1, -0.6207793951034546
#coeffs = np.load(r'G:\Mixed_Reality\cs\nerfstudio\plane_coefficients.npy')
coeffs1 = np.load(r'plane_coefficients.npy')
coeffs2 = np.load(r'plane_coefficients_cs.npy')

a1, b1, c1, d1 = coeffs1
a2, b2, c2, d2 = coeffs2
aabb = np.load(r'aabb.npy')
min_corner = aabb[0]
max_corner = aabb[1]
# Compute the vertices of the AABB
vertices = np.array([
    [min_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], max_corner[1], max_corner[2]],
    [min_corner[0], max_corner[1], max_corner[2]],
])

# Define the edges of the AABB
edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],  # bottom edges
    [4, 5], [5, 6], [6, 7], [7, 4],  # top edges
    [0, 4], [1, 5], [2, 6], [3, 7],  # side edges
]

# Plot the edges
edge_traces = [
    go.Scatter3d(
        x=[vertices[pair[0], 0], vertices[pair[1], 0]],
        y=[vertices[pair[0], 1], vertices[pair[1], 1]],
        z=[vertices[pair[0], 2], vertices[pair[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2)
    ) for pair in edges
]

# Create a grid for the plane
xx, yy = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
zz1 = (-a1 * xx - b1 * yy - d1) / c1
zz2 = (-a2 * xx - b2 * yy - d2) / c2

x_values = []
y_values = []
z_values = []
offset = 0
#print(points1)

""" for point in points:
    if offset % 200 == 0:
        x_values.append(point[0])
        y_values.append(point[1])
        z_values.append(point[2])
    offset += 1

scatter = go.Scatter3d(x=x_values, y=y_values, z=z_values, mode='markers', marker=dict(size=5)) """

scatter1 = go.Scatter3d(x=points1[:,0], y=points1[:,1], z=points1[:,2], mode='markers', marker=dict(size=2))
scatter2 = go.Scatter3d(x=points2[:,0], y=points2[:,1], z=points2[:,2], mode='markers', marker=dict(size=2))

# Plot plane
plane1 = go.Surface(x=xx, y=yy, z=zz1, opacity=0.5)
plane2 = go.Surface(x=xx, y = yy, z = zz2, opacity=0.6)

# Plot bounding box
# bbox_edges = [
#     go.Scatter3d(x=bbox_vertices[[i, j], 0], y=bbox_vertices[[i, j], 1], z=bbox_vertices[[i, j], 2], mode='lines', line=dict(color='red'))
#     for i in range(len(bbox_vertices)) for j in range(i+1, len(bbox_vertices))
# ]

# plot aabb (axis aligned bounding box)


# Combine all plots
fig = go.Figure(data=[scatter1, scatter2, plane1, plane2, occupancy] + edge_traces)
#fig = go.Figure(data=[plane2, occupancy] + edge_traces)

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1,1]),
        yaxis=dict(nticks=4, range=[-1,1]),
        zaxis=dict(nticks=4, range=[-1,1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


In [15]:
import torch
rays_o = torch.Tensor([[1,2,3],[0,0,0]])
print(rays_o.shape)
rays_o = rays_o.T
t = torch.Tensor([1,2])
print(t.shape)
print((rays_o * t).T)

torch.Size([2, 3])
torch.Size([2])
tensor([[1., 2., 3.],
        [0., 0., 0.]])


In [13]:
a = torch.Tensor([1,2,3,4,5,6])
a = a.reshape(-1,3)
print(a.shape)

torch.Size([2, 3])


In [20]:
import numpy as np
import torch
import nerfstudio.utils.math 
import itertools
import math
from dataclasses import dataclass
from typing import Literal, Tuple

import torch
from jaxtyping import Bool, Float
from torch import Tensor

from nerfstudio.data.scene_box import OrientedBox
import plotly.graph_objects as go

def safe_normalize(
    vectors: Float[Tensor, "*batch_dim N"],
    eps: float = 1e-10,
) -> Float[Tensor, "*batch_dim N"]:
    """Normalizes vectors.

    Args:
        vectors: Vectors to normalize.
        eps: Epsilon value to avoid division by zero.

    Returns:
        Normalized vectors.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True) + eps)

def intersect_plane(
    plane_coefficients,     
    origins: torch.Tensor,
    directions: torch.Tensor,
):
    a = plane_coefficients[0]
    b = plane_coefficients[1]
    c = plane_coefficients[2]
    d = plane_coefficients[3]
    normal = torch.Tensor([a,b,c])
    normal = safe_normalize(normal)     #get plane normal
    point = torch.Tensor([0,0,-d/c])
    t = torch.inner((point - origins),normal)/torch.inner(directions,normal)
    return t

object_tensor_aabb= [1,2,3]
rays_o = torch.Tensor([[1,2,3],[0,0,0],[0,1,2],[2,-1,3],[5,7,1],[3,6,9]])
rays_d = torch.Tensor([[0,-1,0],[3,1,2],[-1,0,1],[-1,2,3],[5,1,6],[-2,3,1]])
plane_coefficients = np.load('plane_coefficients.npy')
a = plane_coefficients[0]
b = plane_coefficients[1]
c = plane_coefficients[2]
d = plane_coefficients[3]
t_plane = intersect_plane(plane_coefficients, rays_o, rays_d)
print(t_plane.shape)
print(rays_o.shape)
points = (rays_o.T + rays_d.T * t_plane).T
scatter = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=2))
xx, yy = np.meshgrid(np.linspace(-50, 50, 100), np.linspace(-50, 50, 100))
zz = (-a * xx - b * yy - d) / c
plane = go.Surface(x=xx, y = yy, z = zz, opacity=0.6)
fig = go.Figure(data=[scatter,plane])

fig.show()

torch.Size([6])
torch.Size([6, 3])
